<a href="https://colab.research.google.com/github/shivamrai009/real-time-stock-price-prediction-/blob/main/stockprices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance --quiet
!pip install matplotlib scikit-learn pandas numpy tensorflow --quiet


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [3]:
stock = 'TCS.NS'  # NSE symbol; change this for other stocks
data = yf.download(stock, start='2018-01-01', end='2023-12-31')
data = data[['Open', 'Close']]
data.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Open,Close
Ticker,TCS.NS,TCS.NS
Date,,
2018-01-01,1163.332309,1147.415283
2018-01-02,1153.660829,1141.170044
2018-01-03,1141.517013,1144.401123
2018-01-04,1149.323714,1152.403076
2018-01-05,1149.324017,1166.325317


In [4]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

def create_sequences(data, input_len=60, output_len=3):
    X, y = [], []
    for i in range(len(data) - input_len - output_len):
        X.append(data[i:i+input_len])
        y.append(data[i+input_len:i+input_len+output_len, 1])  # Only 'Close' for prediction
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data)


In [5]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2])),
    Dense(3)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - loss: 0.1082 - val_loss: 0.0013
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0024 - val_loss: 0.0013
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0011 - val_loss: 0.0017
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0010 - val_loss: 9.6939e-04
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 9.5065e-04 - val_loss: 0.0011
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 8.9822

In [7]:
# Recover actual prices
dummy_open = np.zeros((3, 1))
predicted_scaled = predicted_scaled.reshape(3, 1)  # Ensure shape is (3, 1)
combined = np.hstack((dummy_open, predicted_scaled))
predicted_prices = scaler.inverse_transform(combined)[:, 1]
print("Predicted Closing Prices (Next 3 Days):", predicted_prices)


Predicted Closing Prices (Next 3 Days): [3707.01793333 3694.47750833 3698.11692809]
